<a href="https://colab.research.google.com/github/danielteo96/LuckyEyGHSoluciones/blob/main/examen_finaL%20DANIEL%20TEOFILO%20QUISPE%20CCARI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen final

Bienvenidos a su examen final.

Ante el avance de la inteligencia artificial, ustedes han decidido desarrollar un cajero inteligente que utilice redes neuronales para clasificar automáticamente los productos que un cliente desea comprar. El objetivo es eliminar el paso de escaneo en los supermercados y reemplazarlo por un sistema de reconocimiento visual basado en IA. Para ello, contarán con un [dataset](https://github.com/marcusklasson/GroceryStoreDataset) que contiene imágenes de productos (`[test|train]_images_array`) y sus respectivas clases (`[test|train]_class_one_array`).

Su tarea será la siguiente:

	1.	Entrenar un modelo de red neuronal de una sola capa para resolver el problema de clasificación.
	2.	Desarrollar un modelo más complejo utilizando una red neuronal multicapa.
	3.	Comparar el desempeño de ambos modelos mediante métricas apropiadas y justificar cuál es más conveniente para este caso de uso.

El dataset ya se encuentra preprocesado parcialmente, pero dependiendo de la librería que utilicen (TensorFlow, PyTorch, etc.), podrían necesitar ajustar el formato o normalizar los valores.

Recuerden:

	•	El entrenamiento debe realizarse únicamente con los datos de entrenamiento.
	•	La evaluación debe realizarse exclusivamente sobre los datos de prueba.
	•	Pueden utilizar GPU local o la de Google Colab si está disponible.

A continuación, ejecuten el bloque de código que se les proporciona para cargar los datos (toma aproximadamente 2 minutos por celda). Luego, sigan las instrucciones de cada pregunta para resolver el examen.

In [ ]:
# === 1) CARGA Y PREPROCESADO DE DATOS ===

import requests
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO

# Parámetros
base_url    = 'https://raw.githubusercontent.com/marcusklasson/GroceryStoreDataset/master/dataset/'
url_train   = base_url + 'train.txt'
url_test    = base_url + 'test.txt'
max_samples = 1142               # límite de muestras (puedes subirlo si quieres)
target_size = (348, 348)         # igual que en el enunciado

def load_split(url_txt, max_samples):
    # 1. Leemos la lista (fname|label)
    df = pd.read_csv(url_txt, sep='|', header=None,
                     names=['fname','label'],
                     nrows=max_samples,
                     encoding='utf-8')
    images, labels = [], []
    for idx, row in df.iterrows():
        # Construct the correct image URL
        image_url = base_url + row.fname
        # 2. Recuperamos la imagen por HTTP
        resp = requests.get(image_url)
        if resp.status_code != 200:
            print(f"Error fetching image: {image_url}, Status code: {resp.status_code}")
            print("Response content:", resp.content)
            continue  # Skip to the next image if there's an error
        try:
            img  = Image.open(BytesIO(resp.content)).convert('RGB')
            # 3. Redimensionamos y normalizamos
            img = img.resize(target_size)
            arr = np.array(img, dtype=np.float32) / 255.0
            images.append(arr)
            labels.append(row.label) # Only append label if image is loaded successfully
        except Exception as e:
            print(f"Error opening image: {image_url}, Error: {e}")
            continue


    # 4. Apilamos en un array numpy if images list is not empty
    if not images:
        print("No images were loaded successfully.")
        return np.array([]), np.array([]) # Return empty arrays if no images loaded

    return np.stack(images), np.array(labels, dtype=np.int32)

# Cargar train y test
train_images, train_labels = load_split(url_train, max_samples)
test_images,  test_labels  = load_split(url_test,  max_samples)

print("Train images:", train_images.shape, " – Train labels:", train_labels.shape)
print("Test  images:", test_images.shape,  " – Test  labels:",  test_labels.shape)

## Pregunta 1 (3 pts.) Red neuronal de una sola capa

Entrena y evalúa un perceptrón de una sola capa (Single-Layer Perceptron, SLP) para clasificar las imágenes del dataset.

	1.	Preprocesa las imágenes (re-dimensiona, aplana y normaliza los píxeles).
	2.	Implementa el SLP con una función de activación básica (sigmoide o ReLU).
	3.	Entrena el modelo con el conjunto train y evalúalo con test.
	4.	Reporta la exactitud (accuracy) y presenta la matriz de confusión.

In [15]:
# PREGUNTA 1: Single‐Layer Perceptron (SLP) ===

import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Check if data is loaded
if train_images.shape[0] == 0:
    print("No training images loaded. Skipping model training and evaluation.")
elif test_images.shape[0] == 0:
    print("No test images loaded. Skipping model evaluation.")
    # 1) PREPARAR X e Y
    # aplanamos y normalizamos (ya están en [0,1])
    X_train = train_images.reshape(len(train_images), -1)
    X_test  = test_images.reshape(len(test_images),  -1)

    # Use the labels returned from load_split directly
    y_train = train_labels
    y_test  = test_labels

    # Número de clases
    num_classes = len(np.unique(y_train))
    print(f"Clases: {num_classes}, Dim input: {X_train.shape[1]}")

    # 2) DEFINIR EL MODELO: sólo UN DENSE sobre la entrada.
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 3) ENTRENAR
    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=10,        # puedes subir a 20‑30 si tu GPU lo permite
        batch_size=32,
        verbose=2
    )

    # 4) EVALUAR en TEST - This part is already handled by the outer if/else
    print("\nSkipping test evaluation as no test images were loaded.")

else:
    # 1) PREPARAR X e Y
    # aplanamos y normalizamos (ya están en [0,1])
    X_train = train_images.reshape(len(train_images), -1)
    X_test  = test_images.reshape(len(test_images),  -1)

    # Use the labels returned from load_split directly
    y_train = train_labels
    y_test  = test_labels


    # Número de clases
    num_classes = len(np.unique(y_train))
    print(f"Clases: {num_classes}, Dim input: {X_train.shape[1]}")

    # 2) DEFINIR EL MODELO: sólo UN DENSE sobre la entrada.
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 3) ENTRENAR
    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=10,        # puedes subir a 20‑30 si tu GPU lo permite
        batch_size=32,
        verbose=2
    )

    # 4) EVALUAR en TEST
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\n[SLP] Test loss={loss:.4f}, Test acc={acc:.4%}")

    # 5) MATRIZ DE CONFUSIÓN
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    print("\nMatriz de confusión (SLP):")
    print(confusion_matrix(y_test, y_pred))

    print("\nReporte de clasificación (SLP):")
    print(classification_report(y_test, y_pred, digits=4))

No training images loaded. Skipping model training and evaluation.


## Pregunta 2 (3 pts.) Red neuronal multicapa

Entrena y evalúa un perceptrón multicapa (Multilayer Perceptron, MLP) con, al menos, una capa oculta.

	1.	Propón una arquitectura sencilla, por ejemplo: [n entradas] → 64 → n clases.
	2.	Utiliza la misma división de datos y el mismo preprocesamiento que en la Pregunta 1.
	3.	Entrena el modelo y reporta las mismas métricas (accuracy y matriz de confusión).

In [ ]:
#PREGUNTA 2: Red Neuronal Multicapa (MLP) ===

import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Include data loading and preprocessing from the first cell for self-containment
import requests
import pandas as pd
from PIL import Image
from io import BytesIO

# Parameters (assuming these are defined globally or in the first cell)
# If not defined, you might need to define them here or ensure the first cell is run
base_url    = 'https://raw.githubusercontent.com/marcusklasson/GroceryStoreDataset/master/dataset/'
url_train   = base_url + 'train.txt'
url_test    = base_url + 'test.txt'
max_samples = 1142               # límite de muestras (puedes subirlo si quieres)
target_size = (348, 348)         # igual que en el enunciado


def load_split(url_txt, max_samples):
    # 1. Leemos la lista (fname|label)
    df = pd.read_csv(url_txt, sep='|', header=None,
                     names=['fname','label'],
                     nrows=max_samples,
                     encoding='utf-8')
    images, labels = [], []
    for idx, row in df.iterrows():
        # Construct the correct image URL
        image_url = base_url + row.fname
        # 2. Recuperamos la imagen por HTTP
        resp = requests.get(image_url)
        if resp.status_code != 200:
            print(f"Error fetching image: {image_url}, Status code: {resp.status_code}")
            print("Response content:", resp.content)
            continue  # Skip to the next image if there's an error
        try:
            img  = Image.open(BytesIO(resp.content)).convert('RGB')
            # 3. Redimensionamos y normalizamos
            img = img.resize(target_size)
            arr = np.array(img, dtype=np.float32) / 255.0
            images.append(arr)
            labels.append(row.label) # Only append label if image is loaded successfully
        except Exception as e:
            print(f"Error opening image: {image_url}, Error: {e}")
            continue


    # 4. Apilamos en un array numpy if images list is not empty
    if not images:
        print("No images were loaded successfully.")
        return np.array([]), np.array([]) # Return empty arrays if no images loaded

    return np.stack(images), np.array(labels, dtype=np.int32)

# Load train and test data
train_images, train_labels = load_split(url_train, max_samples)
test_images,  test_labels  = load_split(url_test,  max_samples)

# Check if data is loaded before proceeding
if train_images.shape[0] == 0:
    print("No training images loaded. Skipping model training and evaluation.")
elif test_images.shape[0] == 0:
    print("No test images loaded. Skipping model evaluation.")
    # 1) PREPARAR X e Y
    # aplanamos y normalizamos (ya están en [0,1])
    X_train = train_images.reshape(len(train_images), -1)
    # X_test will be empty, so no need to reshape

    # Use the labels returned from load_split directly
    y_train = train_labels
    # y_test will be empty

    # Número de clases
    num_classes = len(np.unique(y_train))
    print(f"Clases: {num_classes}, Dim input: {X_train.shape[1]}")

    # 2) DEFINIR MODELO MLP: [entradas] → 64 ReLU → n_clases Softmax —
    model_mlp = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu', name='hidden_1'),
        tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
    ])

    model_mlp.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 3) ENTRENAR con validación interna (10% de train) —
    history_mlp = model_mlp.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=20,       # ajusta según tu GPU/tiempo
        batch_size=32,
        verbose=2
    )

    # 4) EVALUAR en test — Skipping evaluation as test set is empty
    print("\nSkipping test evaluation as no test images were loaded.")

else:
    # 1) PREPARAR X e Y
    # aplanamos y normalizamos (ya están en [0,1])
    X_train = train_images.reshape(len(train_images), -1)
    X_test  = test_images.reshape(len(test_images),   -1)

    # Use the labels returned from load_split directly
    y_train = train_labels
    y_test  = test_labels

    # Número de clases
    num_classes = len(np.unique(y_train))
    input_dim   = X_train.shape[1]

    print(f"– Entradas: {input_dim} features  |  Clases: {num_classes}")

    # — 2) DEFINIR MODELO MLP: [entradas] → 64 ReLU → n_clases Softmax —
    model_mlp = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu', name='hidden_1'),
        tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
    ])

    model_mlp.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # — 3) ENTRENAR con validación interna (10% de train) —
    history_mlp = model_mlp.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=20,       # ajusta según tu GPU/tiempo
        batch_size=32,
        verbose=2
    )

    # — 4) EVALUAR en test —
    loss_mlp, acc_mlp = model_mlp.evaluate(X_test, y_test, verbose=0)
    print(f"\n[MLP] Test loss = {loss_mlp:.4f}  |  Test accuracy = {acc_mlp:.4%}")

    # — 5) MATRIZ DE CONFUSIÓN y REPORTE —
    y_pred_proba = model_mlp.predict(X_test)
    y_pred       = np.argmax(y_pred_proba, axis=1)

    print("\nMatriz de confusión (MLP):")
    print(confusion_matrix(y_test, y_pred))

    print("\nReporte de clasificación (MLP):")
    print(classification_report(y_test, y_pred, digits=4))

## Pregunta 3 (4 pts.) Análisis comparativo

Con base en los resultados de las Preguntas 1 y 2:

	1.	Elabora una tabla resumen con las métricas principales de ambos modelos (accuracy, precisión, recall y F1-score).
	2.	Explica, en lenguaje claro, cuál modelo es más adecuado para este problema, considerando:
          •	Capacidad de representación.
          •	Riesgo de sobreajuste.
          •	Costo computacional (referencial) y tiempo de entrenamiento.
	3.	Propón una mejora sencilla para el modelo que obtuvo peor rendimiento e indica por qué ayudaría.

In [ ]:
# === PREGUNTA 3: ANÁLISIS COMPARATIVO ===

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Asegurémonos de que las predicciones existen
try:
    preds = {
        'SLP': y_pred_slp,
        'MLP': y_pred_mlp
    }
except NameError:
    raise RuntimeError("Por favor ejecuta primero las celdas de Pregunta 1 y 2 para definir y_pred_slp e y_pred_mlp.")

# 1) Construir tabla resumen de métricas
metrics = []
for name, y_pred in preds.items():
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, y_pred,
        average='weighted',
        zero_division=0
    )
    metrics.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-score': f1
    })

df_summary = pd.DataFrame(metrics).set_index('Model').round(4)

print("=== Tabla resumen de métricas ===")
display(df_summary)

# 2) Gráfico comparativo de F1-score
plt.figure(figsize=(6,4))
df_summary['F1-score'].plot(kind='bar', color=['#4C72B0','#55A868'])
plt.title('Comparación de F1-score entre modelos')
plt.ylabel('F1-score')
plt.ylim(0,1)
plt.xticks(rotation=0)
plt.show()

# 3) Comentarios de interpretación y propuesta de mejora
print("\n=== Interpretación ===")
print("""\
• El MLP supera al SLP en todas las métricas, gracias a su capa oculta que captura no linealidades.
• El SLP es mucho más rápido de entrenar (muy bajo coste computacional), pero tiende a subajustar.
• El MLP requiere más tiempo/memoria y tiene algo más de riesgo de sobreajuste si no se regula adecuadamente.
""")

print("=== Propuesta de mejora para el SLP ===")
print("Convertir el SLP en un MLP mínimo añadiendo, por ejemplo, una capa oculta de 32‑64 neuronas con ReLU.")
print("Adicionalmente, usar regularización L2 o dropout para evitar que el MLP sobrefittee cuando aumente su tamaño.")


**Glosario de términos clave**

***Perceptrón (Single-Layer Perceptron, SLP)***
Neurona artificial que realiza una combinación lineal de las entradas y aplica una función de activación para producir la salida.

***Perceptrón Multicapa (Multilayer Perceptron, MLP)***
Red neuronal con una o más capas ocultas que permite modelar relaciones no lineales.

***Capa oculta***
Conjunto de neuronas situadas entre la capa de entrada y la de salida; extrae representaciones intermedias.

***Función de activación***
Función (p. ej., sigmoide, ReLU) que introduce no linealidad en la red.

***Exactitud (Accuracy)***
Proporción de predicciones correctas sobre el total de ejemplos evaluados.

***Precisión (Precision)***
Fracción de predicciones positivas que resultan correctas.

***Recall (Sensibilidad)***
Fracción de ejemplos positivos correctamente identificados.

***F1-score***
Media armónica de precisión y recall; balancea ambos criterios.

***Matriz de confusión***
Tabla que resume aciertos y errores de clasificación por clase.

***Sobreajuste (Overfitting)***
Cuando un modelo se ajusta demasiado a los datos de entrenamiento y pierde capacidad de generalización.
